In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 

In [30]:
filepath = '/Users/thananpornsethjinda/Desktop/internship/ml/classification/titanic/train.csv'

raw_training_data = pd.read_csv(filepath)

raw_training_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [32]:
filepath = '/Users/thananpornsethjinda/Desktop/internship/ml/classification/titanic/test.csv'

raw_test_data = pd.read_csv(filepath)

raw_test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## Understanding and Conceptualising the Data

In [33]:
raw_training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [34]:
fig = px.histogram(raw_training_data, x='Sex', color='Survived')

fig.show()

Insights: the survival rate of males is lower than the survival rate of females 

In [35]:
fig = px.histogram(raw_training_data, x='Pclass', color='Survived')

fig.show()

Insights: seems like the survival rate of those in the lower case also seem to be higher for some reason

In [36]:
fig = px.histogram(raw_training_data, x='Age', color='Survived')

fig.show()

In [37]:
fig = px.histogram(raw_training_data, x='SibSp', color='Survived')

fig.show()

In [38]:
fig = px.histogram(raw_training_data, x='Embarked', color='Survived')

fig.show()

## Data Cleaning

In [18]:
## data cleaning frame work 

def clean_dataframe(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:
    df = df.copy()

    # log helper
    def log(msg):
        if verbose:
            print(f"[INFO] {msg}")

    # 1. standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    log("Standardized column names.")

    # 2. remove exact duplicates
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        df.drop_duplicates(inplace=True)
        log(f"Removed {dup_count} duplicate rows.")

    # 3. trim and lowercase all string (object) values
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].astype(str).str.strip().str.lower()
    log("Standardized string columns (lowercase + trimmed).")

    # 4. detect missing values (including blanks and placeholders)
    placeholder_values = ['n/a', 'na', '--', '-', 'none', 'null', '', 'nan']
    df.replace(placeholder_values, np.nan, inplace=True)
    null_report = df.isnull().sum()
    null_report = null_report[null_report > 0]
    if not null_report.empty:
        log(f"Missing values found in columns:\n{null_report}")

    # 5. flag constant columns
    constant_cols = [col for col in df.columns if df[col].nunique() == 1]
    if constant_cols:
        log(f"Constant columns (consider removing): {constant_cols}")

    # 6. flag high cardinality categorical columns
    high_card_cols = [col for col in df.select_dtypes(include='object') if df[col].nunique() > 100]
    if high_card_cols:
        log(f"High-cardinality columns (consider encoding strategies): {high_card_cols}")

    # 7. detect numeric outliers using IQR
    num_cols = df.select_dtypes(include=np.number).columns
    outlier_report = {}
    for col in num_cols:
        q1, q3 = df[col].quantile([0.25, 0.75])
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        outliers = df[(df[col] < lower) | (df[col] > upper)][col].count()
        if outliers > 0:
            outlier_report[col] = outliers
    if outlier_report:
        log(f"Potential numeric outliers detected:\n{outlier_report}")
    
    # 8. convert applicable columns to category
    for col in df.select_dtypes(include='object'):
        n_unique = df[col].nunique()
        if n_unique < len(df) * 0.05:
            df[col] = df[col].astype('category')
    log("Converted suitable object columns to category dtype.")

    log("Data cleaning complete.")
    return df

## study this function later 

In [40]:
cleaned_training_data = clean_dataframe(raw_training_data)

[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age         177
cabin       687
embarked      2
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket', 'cabin']
[INFO] Potential numeric outliers detected:
{'age': np.int64(11), 'sibsp': np.int64(46), 'parch': np.int64(213), 'fare': np.int64(116)}
[INFO] Converted suitable object columns to category dtype.
[INFO] Data cleaning complete.


In [41]:
cleaned_training_data

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"braund, mr. owen harris",male,22.0,1,0,a/5 21171,7.2500,NaN,s
1,2,1,1,"cumings, mrs. john bradley (florence briggs th...",female,38.0,1,0,pc 17599,71.2833,c85,c
2,3,1,3,"heikkinen, miss. laina",female,26.0,0,0,ston/o2. 3101282,7.9250,NaN,s
3,4,1,1,"futrelle, mrs. jacques heath (lily may peel)",female,35.0,1,0,113803,53.1000,c123,s
4,5,0,3,"allen, mr. william henry",male,35.0,0,0,373450,8.0500,NaN,s
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"montvila, rev. juozas",male,27.0,0,0,211536,13.0000,NaN,s
887,888,1,1,"graham, miss. margaret edith",female,19.0,0,0,112053,30.0000,b42,s
888,889,0,3,"johnston, miss. catherine helen ""carrie""",female,NaN,1,2,w./c. 6607,23.4500,NaN,s
889,890,1,1,"behr, mr. karl howell",male,26.0,0,0,111369,30.0000,c148,c


In [42]:
cleaned_training_data = cleaned_training_data.drop(['ticket', 'fare', 'cabin', 'name', 'passengerid','sibsp', 'parch'], axis=1)

In [43]:
cleaned_training_data

,survived,pclass,sex,age,embarked
0,0,3,male,22.0,s
1,1,1,female,38.0,c
2,1,3,female,26.0,s
3,1,1,female,35.0,s
4,0,3,male,35.0,s
...,...,...,...,...,...
886,0,2,male,27.0,s
887,1,1,female,19.0,s
888,0,3,female,NaN,s
889,1,1,male,26.0,c


In [44]:
cleaned_test_data = clean_dataframe(raw_test_data)

[INFO] Standardized column names.
[INFO] Standardized string columns (lowercase + trimmed).
[INFO] Missing values found in columns:
age       86
fare       1
cabin    327
dtype: int64
[INFO] High-cardinality columns (consider encoding strategies): ['name', 'ticket']
[INFO] Potential numeric outliers detected:
{'age': np.int64(2), 'sibsp': np.int64(11), 'parch': np.int64(94), 'fare': np.int64(55)}
[INFO] Converted suitable object columns to category dtype.
[INFO] Data cleaning complete.


In [ ]:
cleaned_test_data = cleaned_test_data.drop(['ticket', 'fare', 'cabin', 'name', 'passengerid','sibsp', 'parch'], axis=1)

In [49]:
cleaned_test_data

,pclass,sex,age,embarked
0,3,male,34.5,q
1,3,female,47.0,s
2,2,male,62.0,q
3,3,male,27.0,s
4,3,female,22.0,s
...,...,...,...,...
413,3,male,NaN,s
414,1,female,39.0,c
415,3,male,38.5,s
416,3,male,NaN,s


## Preprocessing 

In [56]:
# imputation of ages 

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

cleaned_training_data[['age']] = imputer.fit_transform(cleaned_training_data[['age']])

cleaned_test_data[['age']] = imputer.transform(cleaned_test_data[['age']])

In [ ]:
# impute embarked column

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='constant', fill_value='unknown')

cleaned_training_data[['embarked']] = imputer.fit_transform(cleaned_training_data[['embarked']])

survived    0
pclass      0
sex         0
age         0
embarked    0
dtype: int64

In [75]:
# one-hot encoding 

categorical_columns = ['pclass', 'sex', 'embarked']

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()

enc.fit(cleaned_training_data[categorical_columns])

OneHotEncoder()

In [77]:
enc.categories_

[array([1, 2, 3]),
 array(['female', 'male'], dtype=object),
 array(['c', 'q', 's', 'unknown'], dtype=object)]

In [ ]:
encoded_columns = list(enc.get_feature_names_out(categorical_columns))

print(encoded_columns)

['pclass_1', 'pclass_2', 'pclass_3', 'sex_female', 'sex_male', 'embarked_c', 'embarked_q', 'embarked_s', 'embarked_unknown']


In [112]:
cleaned_training_data[encoded_columns] = enc.transform(cleaned_training_data[categorical_columns]).toarray()
cleaned_test_data[encoded_columns] = enc.transform(cleaned_test_data[categorical_columns]).toarray()

In [85]:
cleaned_training_data

,survived,pclass,sex,age,embarked,pclass_1,pclass_2,pclass_3,sex_female,sex_male,embarked_c,embarked_q,embarked_s,embarked_unknown
0,0,3,male,22.000000,s,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,1,female,38.000000,c,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1,3,female,26.000000,s,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,female,35.000000,s,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0,3,male,35.000000,s,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,s,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1,1,female,19.000000,s,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
888,0,3,female,29.699118,s,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
889,1,1,male,26.000000,c,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [90]:
# standardisation of numeric columns 

from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

cleaned_training_data[['age']] = scale.fit_transform(cleaned_training_data[['age']])

cleaned_test_data[['age']] = scale.transform(cleaned_test_data[['age']])

## Train the Model

### Logistic Regression

In [103]:
target = cleaned_training_data['survived']

inputs = cleaned_training_data[encoded_columns + ['age']]


In [101]:
inputs

,pclass_1,pclass_2,pclass_3,sex_female,sex_male,embarked_c,embarked_q,embarked_s,embarked_unknown,age
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,-0.592481
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.638789
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.284663
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.407926
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.407926
...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-0.207709
887,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-0.823344
888,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.000000
889,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-0.284663


In [104]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear')

model.fit(inputs, target)

LogisticRegression(solver='liblinear')

In [108]:
train_preds = model.predict(inputs)

train_preds

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,

In [110]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(target, train_preds)

confusion_matrix(target, train_preds)

array([[464,  85],
       [101, 241]])

#### Testing (LR)

In [113]:
cleaned_test_data

,pclass,sex,age,embarked,pclass_1,pclass_2,pclass_3,sex_female,sex_male,embarked_c,embarked_q,embarked_s,embarked_unknown
0,3,male,0.369449,q,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,3,female,1.331378,s,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2,male,2.485693,q,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,3,male,-0.207709,s,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,3,female,-0.592481,s,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,male,0.000000,s,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
414,1,female,0.715743,c,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
415,3,male,0.677266,s,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
416,3,male,0.000000,s,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [119]:
test_inputs = cleaned_test_data[encoded_columns + ['age']]

In [120]:
test_predictions = model.predict(test_inputs)

In [ ]:
output = pd.DataFrame({'PassengerId': raw_test_data.PassengerId, 'Survived': test_predictions})

output


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [123]:
output.to_csv('submission.csv', index=False)


In [ ]:
raw_training_data['Cabin'].isnull().sum()

np.int64(687)

In [133]:
raw_training_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [139]:
raw_training_data['Name'].str.contains('Bradley').sum()

np.int64(2)

In [140]:
raw_training_data.loc[raw_training_data['Name'].str.contains('Bradley')]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
507,508,1,1,"Bradley, Mr. George (""George Arthur Brayton"")",male,NaN,0,0,111427,26.5500,NaN,S


In [141]:
raw_training_data.loc[raw_training_data['SibSp'] == 1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
...,...,...,...,...,...,...,...,...,...,...,...,...
866,867,1,2,"Duran y More, Miss. Asuncion",female,27.0,1,0,SC/PARIS 2149,13.8583,NaN,C
869,870,1,3,"Johnson, Master. Harold Theodor",male,4.0,1,1,347742,11.1333,NaN,S
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
874,875,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.0000,NaN,C


In [142]:
raw_training_data.loc[raw_training_data['SibSp'] == 0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
